### Wine Blending in Python

In [1]:
import numpy as np
from scipy import optimize
import pandas as pd

In [2]:
vintage_costs = np.array([30, 20, 25, 15])
vintage_supply = np.array([180, 250, 200, 400])
blend_price = np.array([70, 40, 30])

In [3]:
initial_quantity = np.ones((12,))
pd.DataFrame(initial_quantity.reshape(4,3), index=[1,2,3,4], columns=['a', 'b', 'c'])

,a,b,c
1,1.0,1.0,1.0
2,1.0,1.0,1.0
3,1.0,1.0,1.0
4,1.0,1.0,1.0


In [4]:
def objective(quantity):
    '''Objective qunction take an array of 12 quantities and returns negative profit since scipy can only minimize'''
    q_reshaped = quantity.reshape(4,3)
    total_sale_price = np.sum(np.dot(blend_price, q_reshaped.T))
    total_cost_price = np.sum(np.dot(vintage_costs, q_reshaped)) 
    return total_cost_price - total_sale_price

In [5]:
def val(q,i,j):
    '''Helper function to access the right element of quantity array in (4,3) shape'''
    if i == 'all':
        return q.reshape(4,3)[:,j-1]
    elif j == 'all':
        return q.reshape(4,3)[i-1,:]
    else:
        return q.reshape(4,3)[i-1,j-1]

In [6]:
cons1 = {'type': 'ineq', 'fun': lambda q: ( val(q,1,1) + val(q,2,1))/np.sum(val(q,'all',1)) - 0.75 }
cons2 = {'type': 'ineq', 'fun': lambda q: 0.05 - val(q,4,1) /np.sum(val(q,'all',1)) }
cons3 = {'type': 'ineq', 'fun': lambda q: (val(q,2,2) + val(q,3,2))/np.sum(val(q, 'all',2)) - 0.35 }
cons4 = {'type': 'ineq', 'fun': lambda q: (val(q,1,3) + val(q,3,3))/np.sum(val(q, 'all',3)) - 0.5 }
cons5 = {'type': 'ineq', 'fun': lambda q: 0.04 - val(q,4,3)/np.sum(val(q, 'all',3)) }
cons6 = {'type': 'ineq', 'fun': lambda q: vintage_supply[0] - np.sum(val(q, 1, 'all')) }
cons7 = {'type': 'ineq', 'fun': lambda q: vintage_supply[1] - np.sum(val(q, 2, 'all')) }
cons8 = {'type': 'ineq', 'fun': lambda q: vintage_supply[2] - np.sum(val(q, 3, 'all')) }
cons9 = {'type': 'ineq', 'fun': lambda q: vintage_supply[3] - np.sum(val(q, 4, 'all')) }

cons = (cons1, cons2, cons3, cons4, cons5, cons6, cons7, cons8, cons9)

In [7]:
b = [(0,vintage_supply[i]) for i in range(4)]
bnds = (b[0],b[0],b[0],b[1],b[1],b[1],b[2],b[2],b[2],b[3],b[3],b[3])

In [8]:
solution = optimize.minimize(objective, initial_quantity.reshape(12,), method='SLSQP', constraints=cons, bounds=bnds)
solution

     fun: -33274.99999578418
     jac: array([-40., -10.,   0., -50., -20., -10., -45., -15.,  -5., -55., -25.,
       -15.])
 message: 'Optimization terminated successfully.'
    nfev: 449
     nit: 32
    njev: 31
  status: 0
 success: True
       x: array([1.80000000e+02, 1.80532708e-11, 5.56354229e-08, 2.28558111e+02,
       2.14418887e+01, 4.39949925e-08, 1.81502220e+01, 1.81849778e+02,
       3.56724895e-12, 2.24583333e+01, 3.77541667e+02, 1.12108368e-12])

In [9]:
result = pd.DataFrame(solution.x.reshape(4,3), index=[1,2,3,4], columns=['a', 'b', 'c'])
result

,a,b,c
1,180.000000,1.805327e-11,5.563542e-08
2,228.558111,2.144189e+01,4.399499e-08
3,18.150222,1.818498e+02,3.567249e-12
4,22.458333,3.775417e+02,1.121084e-12


In [10]:
result['vintage used'] = result.sum(axis=1)
result

,a,b,c,vintage used
1,180.000000,1.805327e-11,5.563542e-08,180.0
2,228.558111,2.144189e+01,4.399499e-08,250.0
3,18.150222,1.818498e+02,3.567249e-12,200.0
4,22.458333,3.775417e+02,1.121084e-12,400.0


In [11]:
profit = - solution.fun
print('Profit: $%0.2f'%profit)

Profit: $33275.00


In [17]:
result.T.style.format("{:.2f}")

,1,2,3,4
a,180.00,228.56,18.15,22.46
b,0.00,21.44,181.85,377.54
c,0.00,0.00,0.00,0.00
vintage used,180.00,250.00,200.00,400.00


In [18]:
result.index

Int64Index([1, 2, 3, 4], dtype='int64')

In [22]:
result.to_dict()

{'a': {1: 179.99999994442314,
  2: 228.55811128857815,
  3: 18.150221998044486,
  4: 22.45833332804088},
 'b': {1: 1.805327083558419e-11,
  2: 21.441888667557794,
  3: 181.84977800205994,
  4: 377.5416666720752},
 'c': {1: 5.5635422854490785e-08,
  2: 4.399499248748095e-08,
  3: 3.5672489511380555e-12,
  4: 1.1210836803601671e-12},
 'vintage used': {1: 180.0000000000766,
  2: 250.00000000013094,
  3: 200.000000000108,
  4: 400.0000000001172}}

In [23]:
loc = pd.DataFrame({'lat':[1,2,3,4,5], 'long':[3,4,5,6,7]}, index=[chr(_) for _ in range(65,70)])

In [24]:
loc

,lat,long
A,1,3
B,2,4
C,3,5
D,4,6
E,5,7


In [46]:
it = loc.iterrows()

In [47]:
locations = {}
for l in it:
    locations[l[0]] = (l[1].lat, l[1].long)

In [62]:
mat = []
for loc_a in locations:
    row = []
    for loc_b in locations:
        distance = haversine(loc_a, loc_b, unit='mi')
        row.append(distance)
    mat.append(row) 
mat

[[5, 10, 16, 10, 9],
 [12, 2, 10, 12, 10],
 [0, 9, 1, 3, 14],
 [5, 9, 7, 6, 18],
 [13, 10, 7, 5, 7]]

In [63]:
pd.DataFrame(mat, index=locations.keys(), columns=locations.keys())

,A,B,C,D,E
A,5,10,16,10,9
B,12,2,10,12,10
C,0,9,1,3,14
D,5,9,7,6,18
E,13,10,7,5,7
